In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests

logic:
1. at the last second block in current epoch, get the effective stakes in current epoch (effective stakes donot change during the whole epoch, and we cannot get the effective stake info at the last block, so we choose the last second)
2. at the last block in current epoch, get the validator reward (accumulated reward in current block - acculated reward in last block)
3. at the first block in new epoch, get the new effective stakes in new epoch, check the effective stakes changes. get the block reward, check the difference of block reward.
4. check if the effective stake increases, whether the validator reward increases, if the effective stake decreases, whether the validator reward decreases.

Note: at the last block in every epoch, in the beginning, the epos-status will change to eligible / not eligible
the currently in committee will be false
and there is no metrics

In [2]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [3]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [4]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [5]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [39]:
def getStakesAndAprs():
    stakes = dict()
    aprs = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['currently-in-committee'] == True:
            address = i['validator']['address']
            effective_stake = 0
            for j in i['metrics']['by-bls-key']:
                effective_stake += float(j['key']['effective-stake'])
       
            apr = float(i['lifetime']['apr'])
            stakes[address] = effective_stake
            aprs[address] = apr
    return stakes, aprs

In [41]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
            if v - map1[k] != 0:
                map3[k] = v - map1[k]
    return map3

In [42]:
def getEpoch():
    method = 'hmy_getEpoch'
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [48]:
def R11_test():
    print("Test-R11: Earning is proportional to effective stake ")
    num = int(input("enter the number of epoches you want to test: "))
    
    iterations = 0
    while iterations < num:
        print("test " + str(iterations+1) + " will begin ...")
        block = getBlockNumber()
        last_block = getLastBlockOfCurrentEpoch()
        if block == last_block:
            new_block = block+1
            while block < new_block:
                block = getBlockNumber()
        epoch = getEpoch()
        print("current epoch: ", epoch)
        print("current and last block numbers", block, last_block)
        
        second_last_block = last_block - 1
        while block < second_last_block:
            block = getBlockNumber()
        print("second last block in current epoch reached", block, "will begin testing...")
        stakes, aprs = getStakesAndAprs()
        
        # in the last block, we can not get the total effective stakes, no metrics. 
        new_block = block + 2
        while block < new_block:
            block = getBlockNumber()
        print("first block in new epoch reached", block, "will compare the changes")
        new_stakes, new_aprs = getStakesAndAprs()

        apr_diff = diffAndFilter(aprs, new_aprs)
        # get the validators whose effective stake changes
        stake_diff = diffAndFilter(stakes, new_stakes)

        if not stake_diff:
            print("in this iteration, no validators change the effective stake")
        if not apr_diff:
            print("in this iteration, no validators change the apr")
            
        flag = True
        for k,v in stake_diff.items():
            if k in apr_diff:
                if v > 0: 
                    if apr_diff[k] <= 0:
                        flag = False
                        print("Test-R11: Fail")
                        print(k, "effective stake increase: ", v)
                        print("apr doesn't increase, apr changes: ", apr_diff[k])
                if v < 0:
                    if apr_diff[k] >= 0:
                        flag = False
                        print("Test-R11: Fail")
                        print(k, "effective stake decrease: ", v)
                        print("apr doesn't decrease, apr changes: ", apr_diff[k])
                        
        if flag:
            print("Test-R11: Success")
        iterations += 1        

In [49]:
R11_test()

Test-R11: Earning is proportional to effective stake 
enter the number of epoches you want to test: 2
test 1 will begin ...
current epoch:  200
current and last block numbers 7604 7637
second last block in current epoch reached 7636 will begin testing...
first block in new epoch reached 7638 will compare the changes
Test-R11: Fail
one1m6j80t6rhc3ypaumtsfmqwjwp0mrqk9ff50prh effective stake increase:  5.067291250000001e+22
apr doesn't increase, apr changes:  -0.11137496545833869
Test-R11: Fail
one1vzsj3julf0ljcj3hhxuqpu6zvadu488zfrtttz effective stake increase:  7.061587321428572e+22
apr doesn't increase, apr changes:  -0.0025263601953611214
Test-R11: Fail
one197p88sfgrvvqptvysxkxk5frnyzgkfjcqh97yu effective stake increase:  5.05729125e+22
apr doesn't increase, apr changes:  -0.11137496545833869
Test-R11: Fail
one1ugkyk6qyzhr5elxr6a7906uv92ctwtzyavx7j6 effective stake increase:  7.061587321428572e+22
apr doesn't increase, apr changes:  -0.0019789810535986163
Test-R11: Fail
one1grlw2qw7t9